### 네이버 지도 크롤링(포기)

In [49]:
from selenium import webdriver

# chromedriver
# 압축해제한 웹드라이버의 경로와 파일명 지정
driver = webdriver.Chrome()

# Load Page
driver.get(url='https://map.naver.com/v5/search/%EA%B0%95%EB%82%A8%EA%B5%AC%20%EC%A7%80%ED%95%98%EC%B2%A0%EC%97%AD?c=13,0,0,0,dh')
# driver.get(url='https://www.naver.com/')


# 현재 URL을 출력
# print(driver.current_url)

# driver.close()

In [52]:
from selenium.webdriver.common.by import By
import time

names = []
####################################### 태그속 스크롤 내리기 하다가 말았음
driver.switch_to.frame("searchIframe")
# itemlist = driver.find_element_by_class_name("Ryr1F")
itemlist = driver.find_elements(By.CSS_SELECTOR,'.Ryr1F')
# driver.execute_script("arguments[0].scrollBy(0, Y)", itemlist)

driver.execute_script("arguments[0].scrollBy(0,2000)", itemlist)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", itemlist)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", itemlist)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", itemlist)
time.sleep(2)
driver.execute_script("arguments[0].scrollBy(0,2000);", itemlist)
time.sleep(2)

# #스크롤 내리기 이동 전 위치
# scroll_location = driver.execute_script("return document.body.scrollHeight")

# while True:
#     #현재 스크롤의 가장 아래로 내림
#     driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
       
#     #전체 스크롤이 늘어날 때까지 대기
#     time.sleep(2)
    
#     #늘어난 스크롤 높이
#     scroll_height = driver.execute_script("return document.body.scrollHeight")

#     #늘어난 스크롤 위치와 이동 전 위치 같으면(더 이상 스크롤이 늘어나지 않으면) 종료
#     if scroll_location == scroll_height:
#         break
        
#     #같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
#     else:
#         #스크롤 위치값을 수정
#         scroll_location = driver.execute_script("return document.body.scrollHeight")
        
# driver.switch_to.frame("searchIframe")
# contents = driver.find_elements(By.CSS_SELECTOR,'#_pcmap_list_scroll_container li')

# driver.switch_to.default_content()

# for content in contents:
#     name = content.find_element(By.CSS_SELECTOR,'.qbGlu a .C6RjW > .place_bluelink').text
#     names.append(name)

# print(contents)
# print(len(contents))
# names

JavascriptException: Message: javascript error: arguments[0].scrollBy is not a function
  (Session info: chrome=115.0.5790.111)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x008CA813+48355]
	(No symbol) [0x0085C4B1]
	(No symbol) [0x00765358]
	(No symbol) [0x007687A1]
	(No symbol) [0x0076A19B]
	(No symbol) [0x007BD279]
	(No symbol) [0x007AA73C]
	(No symbol) [0x007BC922]
	(No symbol) [0x007AA536]
	(No symbol) [0x007882DC]
	(No symbol) [0x007893DD]
	GetHandleVerifier [0x00B2AABD+2539405]
	GetHandleVerifier [0x00B6A78F+2800735]
	GetHandleVerifier [0x00B6456C+2775612]
	GetHandleVerifier [0x009551E0+616112]
	(No symbol) [0x00865F8C]
	(No symbol) [0x00862328]
	(No symbol) [0x0086240B]
	(No symbol) [0x00854FF7]
	BaseThreadInitThunk [0x74FC00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77057B1E+286]
	RtlGetAppContainerNamedObjectPath [0x77057AEE+238]


### 2,3,7,9,수인분당선 주소데이터 합치기

In [1]:
import pandas as pd

df1 = pd.read_csv("./subway/국가철도공단_수도권2호선_주소데이터_20221122.csv", encoding='cp949')
df2 = pd.read_csv("./subway/국가철도공단_수도권3호선_주소데이터_20221122.csv", encoding='cp949')
df3 = pd.read_csv("./subway/국가철도공단_수도권7호선_주소데이터_20221122.csv", encoding='cp949')
df4 = pd.read_csv("./subway/국가철도공단_수도권9호선_주소데이터_20221122.csv", encoding='cp949')
df5 = pd.read_csv("./subway/국가철도공단_수인선_주소데이터_20221122.csv", encoding='cp949')

merged_df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 249 entries, 0 to 248
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   철도운영기관명  249 non-null    object
 1   선명       249 non-null    object
 2   역명       249 non-null    object
 3   지번주소     238 non-null    object
 4   도로명주소    249 non-null    object
dtypes: object(5)
memory usage: 9.9+ KB


In [2]:
def address_preprocessing(address):
    if address.find('(') != -1:
        index = address.find('(')
        return address[:index]
    return address

merged_df['역명'] = merged_df['역명'].apply(address_preprocessing)
merged_df['역명'].apply(lambda x:x.find(')')).value_counts()

-1    249
Name: 역명, dtype: int64

### 구글지도에서 강남구 지하철역명 크롤링

In [3]:
####### 구글지도로 추출
from selenium import webdriver

# chromedriver
# 압축해제한 웹드라이버의 경로와 파일명 지정
driver = webdriver.Chrome()

# Load Page
# chrome을 띄워 네이버 블로그 페이지를 연다.
driver.get(url='https://www.google.com/search?tbs=lf:1,lf_ui:8&tbm=lcl&sxsrf=AB5stBi3ufXxLs2Pt55qIA8L5vgV_-D1bg:1691236834941&q=%EA%B0%95%EB%82%A8%EA%B5%AC+%EC%A7%80%ED%95%98%EC%B2%A0%EC%97%AD&rflfq=1&num=10&rllag=37507560,127034459,1226&sa=X&ved=2ahUKEwj48ZuWvMWAAxUvgVYBHUshAREQjGp6BAgnEAE&biw=958&bih=1073&dpr=1')
# driver.get(url='https://www.naver.com/')


# 현재 URL을 출력
# print(driver.current_url)


In [4]:
from selenium.webdriver.common.by import By
import time

names = []

contents = driver.find_elements(By.CSS_SELECTOR,'[jscontroller="AtSb"]')

for content in contents:
    name = content.find_element(By.CSS_SELECTOR,'span.OSrXXb').text
    names.append(name)

# print(contents)
# print(len(contents))
# names

driver.find_element(By.CSS_SELECTOR,'[aria-label="Page 2"]').click()

time.sleep(2) 
# 쉬는 타임을 가지지 않으면 제대로 크롤링 안됨
# 위 코드에서 추출한 데이터가 반복되서 추출됨

contents = driver.find_elements(By.CSS_SELECTOR,'[jscontroller="AtSb"]')

for content in contents:
    name = content.find_element(By.CSS_SELECTOR,'span.OSrXXb').text
    names.append(name)

# print(contents)
# print(len(contents))
print(names)
# len(names)

driver.close()

['강남', '강남구청', '언주', '역삼', '선정릉', '압구정', '논현역', '선릉', '삼성', '청담', '매봉', '한티', '개포동', '학여울', '학동', '신사', '양재', '수서', '대청', '신논현', '논현', '대치', '일원', '도곡', '구룡', '강남구청역', '봉은사.삼성1파출소앞', '대모산입구', '지하철2호선강남역', '삼성중앙', '교대역(2호선)', '압구정로데오', '한남', '한강진']


In [5]:
import pandas as pd

df = pd.DataFrame(names, columns=['역명'])
df

,역명
0,강남
1,강남구청
2,언주
3,역삼
4,선정릉
5,압구정
6,논현역
7,선릉
8,삼성
9,청담


In [6]:
df.loc[26, '역명'] = '봉은사'
df.loc[30, '역명'] = '교대'
df

,역명
0,강남
1,강남구청
2,언주
3,역삼
4,선정릉
5,압구정
6,논현역
7,선릉
8,삼성
9,청담


In [7]:
print(df.shape)
(df['역명'].duplicated()==False).shape

(34, 1)


(34,)

In [8]:
df_address_check = pd.merge(merged_df, df, on='역명')
df_address_check.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35 entries, 0 to 34
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   철도운영기관명  35 non-null     object
 1   선명       35 non-null     object
 2   역명       35 non-null     object
 3   지번주소     35 non-null     object
 4   도로명주소    35 non-null     object
dtypes: object(5)
memory usage: 1.6+ KB


In [9]:
df_address_check = df_address_check[df_address_check['도로명주소'].str.contains('강남구')]
df_address_check.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 34
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   철도운영기관명  32 non-null     object
 1   선명       32 non-null     object
 2   역명       32 non-null     object
 3   지번주소     32 non-null     object
 4   도로명주소    32 non-null     object
dtypes: object(5)
memory usage: 1.5+ KB


In [10]:
df_address_check = df_address_check[df_address_check['역명'].duplicated()==False]
print(df_address_check.info())
df_address_check

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 34
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   철도운영기관명  27 non-null     object
 1   선명       27 non-null     object
 2   역명       27 non-null     object
 3   지번주소     27 non-null     object
 4   도로명주소    27 non-null     object
dtypes: object(5)
memory usage: 1.3+ KB
None


,철도운영기관명,선명,역명,지번주소,도로명주소
0,서울교통공사,2호선,강남,서울특별시 강남구 역삼동 858 강남역(2호선),서울특별시 강남구 강남대로 지하396(역삼동)
3,서울교통공사,2호선,삼성,서울특별시 강남구 삼성동 172-66 삼성역(2호선),서울특별시 강남구 테헤란로 지하538(삼성동)
4,서울교통공사,2호선,선릉,서울특별시 강남구 삼성동 172-66 선릉역(2호선),서울특별시 강남구 테헤란로 지하340(삼성동)
6,서울교통공사,2호선,역삼,서울특별시 강남구 역삼동 804 역삼역(2호선),서울특별시 강남구 테헤란로 지하156(역삼동)
7,서울교통공사,3호선,대청,서울특별시 강남구 일원동 700-9 대청역(3호선),서울특별시 강남구 일원로 지하2(일원동)
8,서울교통공사,3호선,대치,서울특별시 강남구 대치동 317-3 대치역(3호선),서울특별시 강남구 남부순환로 지하2952(대치동)
9,서울교통공사,3호선,도곡,서울특별시 강남구 도곡동 339-2 도곡역(3호선),서울특별시 강남구 남부순환로 지하2814(도곡동)
11,서울교통공사,3호선,매봉,서울특별시 강남구 도곡동 179-2 매봉역(3호선),서울특별시 강남구 남부순환로 지하2744(도곡동)
12,서울교통공사,3호선,수서,서울특별시 강남구 수서동 728 수서역(3호선),서울특별시 강남구 광평로 지하270(수서동)
14,서울교통공사,3호선,신사,서울특별시 강남구 신사동 667 신사역(3호선),서울특별시 강남구 도산대로 지하102(신사동)


In [11]:
df_address_check = df_address_check.rename(columns={'선명':'호선'})


In [12]:
df_address_check.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 34
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   철도운영기관명  27 non-null     object
 1   호선       27 non-null     object
 2   역명       27 non-null     object
 3   지번주소     27 non-null     object
 4   도로명주소    27 non-null     object
dtypes: object(5)
memory usage: 1.3+ KB


### 역사마스터정보

In [13]:
df_master = pd.read_csv('./subway/서울시 역사마스터 정보.csv', encoding='cp949')
df_master.head()

,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387


In [14]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역사_ID   768 non-null    int64  
 1   역사명     768 non-null    object 
 2   호선      768 non-null    object 
 3   위도      768 non-null    float64
 4   경도      768 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 30.1+ KB


In [15]:
def name_preprocessing(name):
    if name.find('(') != -1:
        index = name.find('(')
        return name[:index]
    return name
    
df_master['역사명'] = df_master['역사명'].apply(name_preprocessing)
df_master['역사명'].apply(lambda x:x.find(')')).value_counts()

-1    768
Name: 역사명, dtype: int64

In [16]:
df_master = df_master.rename(columns={'역사명':'역명'})
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역사_ID   768 non-null    int64  
 1   역명      768 non-null    object 
 2   호선      768 non-null    object 
 3   위도      768 non-null    float64
 4   경도      768 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 30.1+ KB


In [17]:
df_master_merge = pd.merge(df_master, df_address_check, on='역명')
df_master_merge

,역사_ID,역명,호선_x,위도,경도,철도운영기관명,호선_y,지번주소,도로명주소
0,4307,강남,신분당선,37.496837,127.028104,서울교통공사,2호선,서울특별시 강남구 역삼동 858 강남역(2호선),서울특별시 강남구 강남대로 지하396(역삼동)
1,222,강남,2호선,37.497990,127.027912,서울교통공사,2호선,서울특별시 강남구 역삼동 858 강남역(2호선),서울특별시 강남구 강남대로 지하396(역삼동)
2,4306,신논현,신분당선(연장2),37.504598,127.025060,서울9호선,9호선,서울특별시 강남구 역삼동 800,서울특별시 강남구 봉은사로 102
3,4125,신논현,9호선,37.504598,127.025060,서울9호선,9호선,서울특별시 강남구 역삼동 800,서울특별시 강남구 봉은사로 102
4,4305,논현,신분당선(연장2),37.511093,127.021415,서울교통공사,7호선,서울특별시 강남구 논현동 279-67 논현역(7호선),서울특별시 강남구 학동로 지하102(논현동)
5,2734,논현,7호선,37.511093,127.021415,서울교통공사,7호선,서울특별시 강남구 논현동 279-67 논현역(7호선),서울특별시 강남구 학동로 지하102(논현동)
6,4304,신사,신분당선(연장2),37.516334,127.020114,서울교통공사,3호선,서울특별시 강남구 신사동 667 신사역(3호선),서울특별시 강남구 도산대로 지하102(신사동)
7,327,신사,3호선,37.516334,127.020114,서울교통공사,3호선,서울특별시 강남구 신사동 667 신사역(3호선),서울특별시 강남구 도산대로 지하102(신사동)
8,4129,봉은사,9호선(연장),37.514219,127.060245,서울9호선,9호선,서울특별시 강남구 삼성동 172,서울특별시 강남구 봉은사로 601
9,4128,삼성중앙,9호선(연장),37.513011,127.053282,서울9호선,9호선,서울특별시 강남구 삼성동 111-147,서울특별시 강남구 봉은사로 501


In [18]:
df_master_merge['역명'].unique().shape

(27,)

In [19]:
df_master_merge = df_master_merge[df_master_merge['역명'].duplicated()==False]
df_master_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 35
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   역사_ID    27 non-null     int64  
 1   역명       27 non-null     object 
 2   호선_x     27 non-null     object 
 3   위도       27 non-null     float64
 4   경도       27 non-null     float64
 5   철도운영기관명  27 non-null     object 
 6   호선_y     27 non-null     object 
 7   지번주소     27 non-null     object 
 8   도로명주소    27 non-null     object 
dtypes: float64(2), int64(1), object(6)
memory usage: 2.1+ KB


In [21]:
df_master_merge.to_csv('./subway/강남구지하철역_전_u.csv', index=False, encoding='utf-8')
df_master_merge.to_csv('./subway/강남구지하철역_전_c.csv', index=False, encoding='cp949')

### 나무위키 강남구 크롤링

In [23]:
from selenium import webdriver

# chromedriver
# 압축해제한 웹드라이버의 경로와 파일명 지정
driver = webdriver.Chrome()

# Load Page
# chrome을 띄워 네이버 블로그 페이지를 연다.
driver.get(url='https://namu.wiki/w/%EA%B0%95%EB%82%A8%EA%B5%AC')
# driver.get(url='https://www.naver.com/')


# 현재 URL을 출력
# print(driver.current_url)

# driver.close()

In [24]:
from selenium.webdriver.common.by import By
import time

names = []

contents = driver.find_elements(By.CSS_SELECTOR,'[style="text-align:left;"] > div._9yYyRZaH > a._92fTwR77')

for content in contents:
    name = content.text
    names.append(name)


# print(contents)
# print(len(contents))
print(names)
# len(names)

driver.close()

['수서역', '삼성역', '선릉역', '역삼역', '강남역', '압구정역', '신사역', '매봉역', '도곡역', '대치역', '학여울역', '대청역', '일원역', '수서역', '청담역', '강남구청역', '학동역', '논현역', '신논현역', '언주역', '선정릉역', '삼성중앙역', '봉은사역', '압구정로데오역', '강남구청역', '선정릉역', '선릉역', '한티역', '도곡역', '구룡역', '개포동역', '대모산입구역', '수서역', '신사역', '논현역', '신논현역', '강남역']


In [25]:
import pandas as pd

df = pd.DataFrame(names, columns=['역명'])
df

,역명
0,수서역
1,삼성역
2,선릉역
3,역삼역
4,강남역
5,압구정역
6,신사역
7,매봉역
8,도곡역
9,대치역


In [26]:
df['역명'] = df['역명'].str[:-1]
df

,역명
0,수서
1,삼성
2,선릉
3,역삼
4,강남
5,압구정
6,신사
7,매봉
8,도곡
9,대치


In [27]:
df = pd.DataFrame(df['역명'].unique(), columns=['역명'])
df

,역명
0,수서
1,삼성
2,선릉
3,역삼
4,강남
5,압구정
6,신사
7,매봉
8,도곡
9,대치


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   역명      27 non-null     object
dtypes: object(1)
memory usage: 344.0+ bytes


### 역사마스터정보

In [29]:
df_master = pd.read_csv('./subway/서울시 역사마스터 정보.csv', encoding='cp949')
df_master.head()

,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387


In [30]:
df_master.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역사_ID   768 non-null    int64  
 1   역사명     768 non-null    object 
 2   호선      768 non-null    object 
 3   위도      768 non-null    float64
 4   경도      768 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 30.1+ KB


In [31]:
def name_preprocessing(name):
    if name.find('(') != -1:
        index = name.find('(')
        return name[:index]
    return name
    
df_master['역사명'] = df_master['역사명'].apply(name_preprocessing)
df_master['역사명'].apply(lambda x:x.find(')')).value_counts()

-1    768
Name: 역사명, dtype: int64

In [32]:
df_master = df_master.rename(columns = {'역사명':'역명'})
df_master

,역사_ID,역명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387
...,...,...,...,...,...
763,154,종로5가,1호선,37.570926,127.001849
764,153,종로3가,1호선,37.570406,126.991847
765,152,종각,1호선,37.570161,126.982923
766,151,시청,1호선,37.565715,126.977088


In [33]:
df_master_merge = pd.merge(df_master, df, on='역명')
df_master_merge

,역사_ID,역명,호선,위도,경도
0,4307,강남,신분당선,37.496837,127.028104
1,222,강남,2호선,37.497990,127.027912
2,4306,신논현,신분당선(연장2),37.504598,127.025060
3,4125,신논현,9호선,37.504598,127.025060
4,4305,논현,신분당선(연장2),37.511093,127.021415
5,2734,논현,7호선,37.511093,127.021415
6,4304,신사,신분당선(연장2),37.516334,127.020114
7,327,신사,3호선,37.516334,127.020114
8,4129,봉은사,9호선(연장),37.514219,127.060245
9,4128,삼성중앙,9호선(연장),37.513011,127.053282


In [34]:
df_master_merge['역명'].unique().shape

(27,)

### 지하철 개통 데이터
- 2018년이후에 개통된 역을 확인한다.

In [35]:
import pandas as pd

df3 = pd.read_csv('./subway/서울교통공사_지하철개통정보_20220627.csv', encoding='cp949')
df3 = df3.sort_values('개통일자')
df3.reset_index(drop=True, inplace=True)
df3

# 18년도 이후에 개통된 역중 강남구 지하철역은 포함되어있지 않음

,연번,기관,호선,구간,역개수,연장(km),개통일자
0,1,서울교통공사,1,서울↔청량리,9.0,7.8,1974-08-15
1,3,서울교통공사,2,신설동(2)↔종합운동장,11.0,14.3,1980-10-31
2,4,서울교통공사,2,종합운동장↔교대(2),5.0,5.5,1982-12-23
3,5,서울교통공사,2,을지입구↔성수,9.0,7.9,1983-09-16
4,6,서울교통공사,2,교대(2)↔서울대입구,5.0,6.7,1983-12-17
5,7,서울교통공사,2,서울대입구↔을지입구,16.0,19.8,1984-05-22
6,17,서울교통공사,4,상계↔한성대입구,10.0,11.8,1985-04-20
7,12,서울교통공사,3,구파발↔독립문,7.0,8.0,1985-07-12
8,13,서울교통공사,3,독립문↔양재,16.0,18.2,1985-10-18
9,18,서울교통공사,4,한성대입구↔사당,14.0,16.5,1985-10-18


In [36]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연번      41 non-null     int64  
 1   기관      41 non-null     object 
 2   호선      41 non-null     int64  
 3   구간      41 non-null     object 
 4   역개수     40 non-null     float64
 5   연장(km)  37 non-null     float64
 6   개통일자    41 non-null     object 
dtypes: float64(2), int64(2), object(3)
memory usage: 2.4+ KB


In [37]:
df3['개통일자'] = pd.to_datetime(df3['개통일자'], format='%Y-%m-%d', errors='coerce')
df3['개통일자']

0    1974-08-15
1    1980-10-31
2    1982-12-23
3    1983-09-16
4    1983-12-17
5    1984-05-22
6    1985-04-20
7    1985-07-12
8    1985-10-18
9    1985-10-18
10   1990-07-13
11   1992-05-22
12   1993-04-21
13   1993-10-30
14   1994-04-01
15   1995-11-15
16   1996-02-29
17   1996-03-20
18   1996-03-20
19   1996-03-30
20   1996-08-12
21   1996-10-11
22   1996-11-23
23   1996-12-30
24   1999-07-02
25   2000-02-29
26   2000-08-01
27   2000-08-07
28   2000-12-15
29   2001-03-09
30   2005-10-20
31   2005-12-21
32   2010-02-18
33   2012-10-27
34   2015-03-28
35   2018-12-01
36   2019-12-21
37   2020-08-08
38   2021-03-27
39   2021-12-18
40   2022-03-19
Name: 개통일자, dtype: datetime64[ns]

In [38]:
df3 = df3[df3['개통일자'] >= '20180101']
df3

,연번,기관,호선,구간,역개수,연장(km),개통일자
35,41,서울메트로9호선운영,9,종합운동장(9)↔중앙보훈병원,8.0,9.1,2018-12-01
36,32,서울교통공사,6,봉화산↔신내,1.0,1.3,2019-12-21
37,27,서울교통공사,5,미사↔하남풍산,2.0,4.6,2020-08-08
38,28,서울교통공사,5,강일↔하남검단산,3.0,2.9,2021-03-27
39,39,서울교통공사,8,남위례,1.0,NaN,2021-12-18
40,21,서울교통공사,4,당고개↔진접,3.0,14.2,2022-03-19


In [39]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 35 to 40
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   연번      6 non-null      int64         
 1   기관      6 non-null      object        
 2   호선      6 non-null      int64         
 3   구간      6 non-null      object        
 4   역개수     6 non-null      float64       
 5   연장(km)  5 non-null      float64       
 6   개통일자    6 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 384.0+ bytes


In [40]:
df3['호선'] = df3['호선'].astype(str) + '호선'
df3

C:\Users\user\AppData\Local\Temp\ipykernel_6084\2474712161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['호선'] = df3['호선'].astype(str) + '호선'


,연번,기관,호선,구간,역개수,연장(km),개통일자
35,41,서울메트로9호선운영,9호선,종합운동장(9)↔중앙보훈병원,8.0,9.1,2018-12-01
36,32,서울교통공사,6호선,봉화산↔신내,1.0,1.3,2019-12-21
37,27,서울교통공사,5호선,미사↔하남풍산,2.0,4.6,2020-08-08
38,28,서울교통공사,5호선,강일↔하남검단산,3.0,2.9,2021-03-27
39,39,서울교통공사,8호선,남위례,1.0,NaN,2021-12-18
40,21,서울교통공사,4호선,당고개↔진접,3.0,14.2,2022-03-19


In [41]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6 entries, 35 to 40
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   연번      6 non-null      int64         
 1   기관      6 non-null      object        
 2   호선      6 non-null      object        
 3   구간      6 non-null      object        
 4   역개수     6 non-null      float64       
 5   연장(km)  5 non-null      float64       
 6   개통일자    6 non-null      datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 384.0+ bytes


In [42]:
df4 = pd.merge(df_master_merge, df3, on='호선')
df4

,역사_ID,역명,호선,위도,경도,연번,기관,구간,역개수,연장(km),개통일자
0,4125,신논현,9호선,37.504598,127.02506,41,서울메트로9호선운영,종합운동장(9)↔중앙보훈병원,8.0,9.1,2018-12-01


In [28]:
# 9호선도 직접확인한 결과 고려대상이 아니다.
# Selenium으로 확인할 수 있어보이는데 시간여유가 있으면 해보는걸로..

### 메인데이터와 합치기 - cross join

In [43]:
# help(pd.merge)

#     how : {'left', 'right', 'outer', 'inner', 'cross'}, default 'inner'
#         Type of merge to be performed.
#         * cross: creates the cartesian product from both frames, preserves the order
#           of the left keys.


In [81]:
df_main = pd.read_csv('./main/강남구전세실거래가_전_c.csv', encoding='cp949')
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87361 entries, 0 to 87360
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   전월세구분    87361 non-null  object 
 1   보증금(만원)  87361 non-null  int64  
 2   구분       87361 non-null  object 
 3   층        77889 non-null  float64
 4   면적       87361 non-null  float64
 5   도로명주소    87361 non-null  object 
 6   구주소      87361 non-null  object 
 7   동주소      87361 non-null  object 
 8   건물 나이    86480 non-null  float64
 9   계약년월일    87361 non-null  object 
 10  계약시작     25069 non-null  object 
 11  계약종료     25069 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 8.0+ MB


In [82]:
df_master_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 35
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   역사_ID   36 non-null     int64  
 1   역명      36 non-null     object 
 2   호선      36 non-null     object 
 3   위도      36 non-null     float64
 4   경도      36 non-null     float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.7+ KB


In [84]:
df_join = pd.merge(df_main, df_master_merge, how='cross')
print(df_join.shape)
len(df_main)*len(df_master_merge)
# 두 값이 같은것으로 크로스조인을 확인할 수 있다.

(3144996, 17)


3144996

### 밑은 필요없는 자료임

### 1~8호선 위경도 데이터에서 강남구 지하철역 위경도 데이터 뽑기

In [5]:
df_sub_coor = pd.read_csv('./subway/서울교통공사_1_8호선 역사 좌표(위경도) 정보_20211231.csv', encoding='cp949')
df_sub_coor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            276 non-null    int64  
 1   호선            276 non-null    int64  
 2   고유역번호(외부역코드)  276 non-null    int64  
 3   역명            276 non-null    object 
 4   위도            276 non-null    float64
 5   경도            276 non-null    float64
 6   작성일자          276 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 15.2+ KB


In [6]:
df_merge = pd.merge(df_sub_coor,df2)
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            21 non-null     int64  
 1   호선            21 non-null     int64  
 2   고유역번호(외부역코드)  21 non-null     int64  
 3   역명            21 non-null     object 
 4   위도            21 non-null     float64
 5   경도            21 non-null     float64
 6   작성일자          21 non-null     object 
dtypes: float64(2), int64(3), object(2)
memory usage: 1.3+ KB


### 9호선 위경도데이터 뽑기

#### 강남구 9호선 지하철역 뽑기

In [25]:
import pandas as pd

df_sub_add = pd.read_csv('./subway/서울교통공사 지하철역 주소 및 전화번호 정보.csv', encoding='cp949')
df_sub_add.head()

,호선,역명,구주소,도로명주소,전화번호
0,2호선,봉천,서울특별시 관악구 봉천동 979-7 봉천역(2호선),서울특별시 관악구 남부순환로 지하1721(봉천동),02-6110-2291
1,1호선,서울,서울특별시 중구 남대문로5가 73-6 서울역(1호선),서울특별시 중구 세종대로 지하2(남대문로 5가),02-6110-1331
2,1호선,시청,서울특별시 중구 정동 5-5 시청역(1호선),서울특별시 중구 세종대로 지하101(정동),02-6110-1321
3,1호선,종각,서울특별시 종로구 종로1가 54 종각역(1호선),서울특별시 종로구 종로 지하55(종로1가),02-6110-1311
4,1호선,종로3가,서울특별시 종로구 종로3가 10-5 종로3가역(1호선),서울특별시 종로구 종로 지하129(종로3가),02-6110-1301


#### 도로명 주소 전처리

In [26]:
def address_preprocessing(address):
    index = address.find('(')
    return address[:index]

df_sub_add['도로명주소'] = df_sub_add['도로명주소'].apply(address_preprocessing)
df_sub_add['도로명주소']

0      서울특별시 관악구 남부순환로 지하1721
1           서울특별시 중구 세종대로 지하2
2         서울특별시 중구 세종대로 지하101
3           서울특별시 종로구 종로 지하55
4          서울특별시 종로구 종로 지하129
                ...          
283     서울특별시 송파구 백제고분로 지하446
284      서울특별시 송파구 위례성대로 지하29
285     서울특별시 송파구 양재대로 지하1233
286      서울특별시 강동구 강동대로 지하303
287       서울특별시 강동구 동남로 지하625
Name: 도로명주소, Length: 288, dtype: object

In [9]:
df_sub_add = df_sub_add[(df_sub_add['호선']=='9호선')&(df_sub_add['도로명주소'].str.contains('강남구'))]
print(df_sub_add)
df_sub_add.to_csv('./subway/강남구9호선지하철역.csv', index=False, encoding='cp949')

      호선    역명                               구주소                 도로명주소  \
275  9호선    언주    서울특별시 강남구 논현동 279-165 언주역(9호선)  서울특별시 강남구 봉은사로 지하201   
276  9호선   선정릉   서울특별시 강남구 삼성동 111-114 선정릉역(9호선)   서울특별시 강남구 선릉로 지하580   
277  9호선  삼성중앙  서울특별시 강남구 삼성동 111-147 삼성중앙역(9호선)  서울특별시 강남구 봉은사로 지하501   
278  9호선   봉은사       서울특별시 강남구 삼성동 172 봉은사역(9호선)  서울특별시 강남구 봉은사로 지하601   

             전화번호  
275  02-2656-0926  
276  02-2656-0927  
277  02-2656-0928  
278  02-2656-0929  


In [23]:
print(len(df_merge))
df_merge

21


,연번,호선,고유역번호(외부역코드),역명,위도,경도,작성일자
0,28,2,219,삼성,37.508827,127.063203,1982-12-30
1,29,2,220,선릉,37.504257,127.048174,1982-12-30
2,30,2,221,역삼,37.500658,127.036430,1982-12-30
3,31,2,222,강남,37.497958,127.027539,1982-12-30
4,32,2,223,교대,37.493957,127.014631,1982-12-20
5,83,3,330,교대,37.493060,127.013796,1985-10-18
6,79,3,326,압구정,37.526169,127.028502,1985-10-18
7,80,3,327,신사,37.516438,127.020247,1985-10-18
8,85,3,332,양재,37.484660,127.035130,1985-10-18
9,86,3,333,매봉,37.487114,127.046907,1993-12-31


In [27]:
df_sub_add = df_sub_add[(df_sub_add['호선']!='9호선')&(df_sub_add['도로명주소'].str.contains('강남구'))]
print(len(df_sub_add))
df_sub_add

17


,호선,역명,구주소,도로명주소,전화번호
29,2호선,삼성(무역센터),서울특별시 강남구 삼성동 172-66 삼성역(2호선),서울특별시 강남구 테헤란로 지하538,02-6110-2191
30,2호선,선릉,서울특별시 강남구 삼성동 172-66 선릉역(2호선),서울특별시 강남구 테헤란로 지하340,02-6110-2201
31,2호선,역삼,서울특별시 강남구 역삼동 804 역삼역(2호선),서울특별시 강남구 테헤란로 지하156,02-6110-2211
32,2호선,강남,서울특별시 강남구 역삼동 858 강남역(2호선),서울특별시 강남구 강남대로 지하396,02-6110-2221
77,3호선,압구정,서울특별시 강남구 신사동 668 압구정역(3호선),서울특별시 강남구 압구정로 지하172,02-6110-3361
78,3호선,신사,서울특별시 강남구 신사동 667 신사역(3호선),서울특별시 강남구 도산대로 지하102,02-6110-3371
84,3호선,매봉,서울특별시 강남구 도곡동 179-2 매봉역(3호선),서울특별시 강남구 남부순환로 지하2744,02-6110-3431
85,3호선,도곡,서울특별시 강남구 도곡동 339-2 도곡역(3호선),서울특별시 강남구 남부순환로 지하2814,02-6110-3441
86,3호선,대치,서울특별시 강남구 대치동 317-3 대치역(3호선),서울특별시 강남구 남부순환로 지하2952,02-6110-3451
87,3호선,학여울,서울특별시 강남구 대치동 514-3 학여울역(3호선),서울특별시 강남구 남부순환로 지하3104,02-6110-3461


In [28]:
df_sub_add.loc[29,'역명'] = '삼성'
df_sub_add

,호선,역명,구주소,도로명주소,전화번호
29,2호선,삼성,서울특별시 강남구 삼성동 172-66 삼성역(2호선),서울특별시 강남구 테헤란로 지하538,02-6110-2191
30,2호선,선릉,서울특별시 강남구 삼성동 172-66 선릉역(2호선),서울특별시 강남구 테헤란로 지하340,02-6110-2201
31,2호선,역삼,서울특별시 강남구 역삼동 804 역삼역(2호선),서울특별시 강남구 테헤란로 지하156,02-6110-2211
32,2호선,강남,서울특별시 강남구 역삼동 858 강남역(2호선),서울특별시 강남구 강남대로 지하396,02-6110-2221
77,3호선,압구정,서울특별시 강남구 신사동 668 압구정역(3호선),서울특별시 강남구 압구정로 지하172,02-6110-3361
78,3호선,신사,서울특별시 강남구 신사동 667 신사역(3호선),서울특별시 강남구 도산대로 지하102,02-6110-3371
84,3호선,매봉,서울특별시 강남구 도곡동 179-2 매봉역(3호선),서울특별시 강남구 남부순환로 지하2744,02-6110-3431
85,3호선,도곡,서울특별시 강남구 도곡동 339-2 도곡역(3호선),서울특별시 강남구 남부순환로 지하2814,02-6110-3441
86,3호선,대치,서울특별시 강남구 대치동 317-3 대치역(3호선),서울특별시 강남구 남부순환로 지하2952,02-6110-3451
87,3호선,학여울,서울특별시 강남구 대치동 514-3 학여울역(3호선),서울특별시 강남구 남부순환로 지하3104,02-6110-3461


In [29]:
df_crawling_effect = pd.merge(df_merge, df_sub_add, on='역명', how='left')
df_crawling_effect

,연번,호선_x,고유역번호(외부역코드),역명,위도,경도,작성일자,호선_y,구주소,도로명주소,전화번호
0,28,2,219,삼성,37.508827,127.063203,1982-12-30,2호선,서울특별시 강남구 삼성동 172-66 삼성역(2호선),서울특별시 강남구 테헤란로 지하538,02-6110-2191
1,29,2,220,선릉,37.504257,127.048174,1982-12-30,2호선,서울특별시 강남구 삼성동 172-66 선릉역(2호선),서울특별시 강남구 테헤란로 지하340,02-6110-2201
2,30,2,221,역삼,37.500658,127.036430,1982-12-30,2호선,서울특별시 강남구 역삼동 804 역삼역(2호선),서울특별시 강남구 테헤란로 지하156,02-6110-2211
3,31,2,222,강남,37.497958,127.027539,1982-12-30,2호선,서울특별시 강남구 역삼동 858 강남역(2호선),서울특별시 강남구 강남대로 지하396,02-6110-2221
4,32,2,223,교대,37.493957,127.014631,1982-12-20,NaN,NaN,NaN,NaN
5,83,3,330,교대,37.493060,127.013796,1985-10-18,NaN,NaN,NaN,NaN
6,79,3,326,압구정,37.526169,127.028502,1985-10-18,3호선,서울특별시 강남구 신사동 668 압구정역(3호선),서울특별시 강남구 압구정로 지하172,02-6110-3361
7,80,3,327,신사,37.516438,127.020247,1985-10-18,3호선,서울특별시 강남구 신사동 667 신사역(3호선),서울특별시 강남구 도산대로 지하102,02-6110-3371
8,85,3,332,양재,37.484660,127.035130,1985-10-18,NaN,NaN,NaN,NaN
9,86,3,333,매봉,37.487114,127.046907,1993-12-31,3호선,서울특별시 강남구 도곡동 179-2 매봉역(3호선),서울특별시 강남구 남부순환로 지하2744,02-6110-3431


In [10]:
df_sub_9_coor = pd.read_csv('./subway/강남구9호선지하철역_위경도.csv', encoding='utf-8')
df_sub_9_coor

,호선,역명,구주소,도로명주소,Latitude,Longitude,전화번호
0,9호선,언주,서울특별시 강남구 논현동 279-165 언주역(9호선),서울특별시 강남구 봉은사로 201,37.510948,127.047282,02-2656-0926
1,9호선,선정릉,서울특별시 강남구 삼성동 111-114 선정릉역(9호선),서울특별시 강남구 선릉로 580,37.500678,127.048943,02-2656-0927
2,9호선,삼성중앙,서울특별시 강남구 삼성동 111-147 삼성중앙역(9호선),서울특별시 강남구 봉은사로 501,37.510948,127.047282,02-2656-0928
3,9호선,봉은사,서울특별시 강남구 삼성동 172 봉은사역(9호선),서울특별시 강남구 봉은사로 601,37.510948,127.047282,02-2656-0929


### 1~8호선 9호선 위경도 데이터 합치기(concat)

In [11]:
print(df_merge.info())
df_sub_9_coor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 20
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            21 non-null     int64  
 1   호선            21 non-null     int64  
 2   고유역번호(외부역코드)  21 non-null     int64  
 3   역명            21 non-null     object 
 4   위도            21 non-null     float64
 5   경도            21 non-null     float64
 6   작성일자          21 non-null     object 
dtypes: float64(2), int64(3), object(2)
memory usage: 1.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   호선         4 non-null      object 
 1   역명         4 non-null      object 
 2   구주소        4 non-null      object 
 3   도로명주소      4 non-null      object 
 4   Latitude   4 non-null      float64
 5   Longitude  4 non-null      float64
 6   전화번호       4 non-nu

#### concat을 위한 전처리

In [12]:
print(df_merge['호선'])
df_sub_9_coor['호선']

0     2
1     2
2     2
3     2
4     2
5     3
6     3
7     3
8     3
9     3
10    3
11    3
12    3
13    3
14    3
15    3
16    6
17    7
18    7
19    7
20    7
Name: 호선, dtype: int64


0    9호선
1    9호선
2    9호선
3    9호선
Name: 호선, dtype: object

In [13]:
df_sub_9_coor['호선'] = pd.to_numeric(df_sub_9_coor['호선'].str[0])
df_sub_9_coor['호선']

0    9
1    9
2    9
3    9
Name: 호선, dtype: int64

In [14]:
print(df_merge['역명'])
df_sub_9_coor['역명']

0       삼성
1       선릉
2       역삼
3       강남
4       교대
5       교대
6      압구정
7       신사
8       양재
9       매봉
10      도곡
11      대치
12     학여울
13      대청
14      일원
15      수서
16     한강진
17      청담
18    강남구청
19      학동
20      논현
Name: 역명, dtype: object


0      언주
1     선정릉
2    삼성중앙
3     봉은사
Name: 역명, dtype: object

In [15]:
df_sub_9_coor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   호선         4 non-null      int64  
 1   역명         4 non-null      object 
 2   구주소        4 non-null      object 
 3   도로명주소      4 non-null      object 
 4   Latitude   4 non-null      float64
 5   Longitude  4 non-null      float64
 6   전화번호       4 non-null      object 
dtypes: float64(2), int64(1), object(4)
memory usage: 352.0+ bytes


In [16]:
df_sub_9_coor = df_sub_9_coor.rename(columns={'Latitude':'위도', 'Longitude':'경도'})
df_sub_9_coor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   호선      4 non-null      int64  
 1   역명      4 non-null      object 
 2   구주소     4 non-null      object 
 3   도로명주소   4 non-null      object 
 4   위도      4 non-null      float64
 5   경도      4 non-null      float64
 6   전화번호    4 non-null      object 
dtypes: float64(2), int64(1), object(4)
memory usage: 352.0+ bytes


In [17]:
concat_df = pd.concat([df_merge, df_sub_9_coor], ignore_index=True)
concat_df['호선'].value_counts()

3    11
2     5
7     4
9     4
6     1
Name: 호선, dtype: int64

In [18]:
concat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   연번            21 non-null     float64
 1   호선            25 non-null     int64  
 2   고유역번호(외부역코드)  21 non-null     float64
 3   역명            25 non-null     object 
 4   위도            25 non-null     float64
 5   경도            25 non-null     float64
 6   작성일자          21 non-null     object 
 7   구주소           4 non-null      object 
 8   도로명주소         4 non-null      object 
 9   전화번호          4 non-null      object 
dtypes: float64(4), int64(1), object(5)
memory usage: 2.1+ KB


In [19]:
concat_df = concat_df[['호선','역명','위도','경도','도로명주소']]
concat_df

,호선,역명,위도,경도,도로명주소
0,2,삼성,37.508827,127.063203,NaN
1,2,선릉,37.504257,127.048174,NaN
2,2,역삼,37.500658,127.036430,NaN
3,2,강남,37.497958,127.027539,NaN
4,2,교대,37.493957,127.014631,NaN
5,3,교대,37.493060,127.013796,NaN
6,3,압구정,37.526169,127.028502,NaN
7,3,신사,37.516438,127.020247,NaN
8,3,양재,37.484660,127.035130,NaN
9,3,매봉,37.487114,127.046907,NaN


In [20]:
concat_df.drop('도로명주소', axis=1, inplace=True)
concat_df.drop(5, axis=0, inplace=True) # 교대가 2.3호선 환승역이라 중복되었음
concat_df

,호선,역명,위도,경도
0,2,삼성,37.508827,127.063203
1,2,선릉,37.504257,127.048174
2,2,역삼,37.500658,127.036430
3,2,강남,37.497958,127.027539
4,2,교대,37.493957,127.014631
6,3,압구정,37.526169,127.028502
7,3,신사,37.516438,127.020247
8,3,양재,37.484660,127.035130
9,3,매봉,37.487114,127.046907
10,3,도곡,37.491129,127.055694


In [48]:
concat_df.reset_index(drop=True, inplace=True)
print(concat_df.shape)
concat_df

(24, 4)


,호선,역명,위도,경도
0,2,삼성,37.508827,127.063203
1,2,선릉,37.504257,127.048174
2,2,역삼,37.500658,127.036430
3,2,강남,37.497958,127.027539
4,2,교대,37.493957,127.014631
5,3,압구정,37.526169,127.028502
6,3,신사,37.516438,127.020247
7,3,양재,37.484660,127.035130
8,3,매봉,37.487114,127.046907
9,3,도곡,37.491129,127.055694
